In [ ]:
import sys
sys.path.append('../src')
from vsa import VSA
import numpy as np

vsa = VSA(dim=2048)

# Generate random atomic vectors for concepts
np.random.seed(42)
concepts = [f'concept_{i}' for i in range(100)]  # 50 pairs need 100 concepts
atomic_vectors = {concept: np.random.normal(0, 1/np.sqrt(vsa.dim), vsa.dim) for concept in concepts}

# Create 50 bound facts (e.g., concept_0 bound to concept_1, etc.)
facts = []
for i in range(0, 100, 2):
    role = atomic_vectors[concepts[i]]
    filler = atomic_vectors[concepts[i+1]]
    bound = vsa.bind(role, filler)
    facts.append(bound)

# Bundle all facts into the memory vector M
M = vsa.bundle(facts)

In [ ]:
true_scores = []
false_scores = []

# True queries (25 facts present)
for i in range(0, 50, 2):
    role = atomic_vectors[concepts[i]]
    expected_filler = atomic_vectors[concepts[i+1]]
    retrieved = vsa.unbind(role, M)
    coherence = np.dot(retrieved, expected_filler) / (np.linalg.norm(retrieved) * np.linalg.norm(expected_filler))
    true_scores.append(coherence)

# False queries (25 facts not present)
false_concepts = [f'false_concept_{j}' for j in range(50)]
false_atomic = {c: np.random.normal(0, 1/np.sqrt(vsa.dim), vsa.dim) for c in false_concepts}
for j in range(0, 50, 2):
    role = false_atomic[false_concepts[j]]
    expected_filler = false_atomic[false_concepts[j+1]]
    retrieved = vsa.unbind(role, M)
    coherence = np.dot(retrieved, expected_filler) / (np.linalg.norm(retrieved) * np.linalg.norm(expected_filler))
    false_scores.append(coherence)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(true_scores, bins=20, alpha=0.5, label='True Facts')
plt.hist(false_scores, bins=20, alpha=0.5, label='False Facts')
plt.legend()
plt.xlabel('Coherence Score')
plt.ylabel('Frequency')
plt.title('Distribution of Coherence Scores: True vs. False Facts')
plt.show()